In [8]:
#Imports
import os
import sys
import json
from html.parser import HTMLParser
import glob
import zipfile
from bs4 import BeautifulSoup
import re

In [12]:
os.path.abspath(os.curdir)

'C:\\Users\\cschu\\Ddrivelink\\IntroMLProject\\Code'

In [16]:
os.chdir('../../congress/data')

In [18]:
text_path = os.path.abspath(os.curdir)

In [20]:
#Generate list of bill paths for each congress
hr_paths = []
s_paths = []
for i in range(107,116):
    hr_paths.append(text_path+'\\'+str(i)+'\\bills\\hr')
    s_paths.append(text_path+'\\'+str(i)+'\\bills\\s')

In [21]:
s_paths

['C:\\Users\\cschu\\Ddrivelink\\congress\\data\\107\\bills\\s',
 'C:\\Users\\cschu\\Ddrivelink\\congress\\data\\108\\bills\\s',
 'C:\\Users\\cschu\\Ddrivelink\\congress\\data\\109\\bills\\s',
 'C:\\Users\\cschu\\Ddrivelink\\congress\\data\\110\\bills\\s',
 'C:\\Users\\cschu\\Ddrivelink\\congress\\data\\111\\bills\\s',
 'C:\\Users\\cschu\\Ddrivelink\\congress\\data\\112\\bills\\s',
 'C:\\Users\\cschu\\Ddrivelink\\congress\\data\\113\\bills\\s',
 'C:\\Users\\cschu\\Ddrivelink\\congress\\data\\114\\bills\\s',
 'C:\\Users\\cschu\\Ddrivelink\\congress\\data\\115\\bills\\s']

# Unzip compressed folders with bill text versions

In [22]:
#Only use original version of bill
#Currently don't know how to obtain latest bill text version

In [23]:
#Function to create list of zip folders
def create_zip_folders(path,version_type):
    zip_folders=[]
    for p in path:
        zip_folders.append(glob.glob(p+'/**/text-versions/'+version_type+'/package.zip', recursive=True))
    
    #Flatten list of lists
    zip_folders=[item for sublist in zip_folders for item in sublist]
    
    return zip_folders

In [24]:
#Function to unzip bill text folder for specific version

def unzip_text_folder(zip_folders):
    for f in zip_folders:
        extract_path = f.replace('\\package.zip','')
        with zipfile.ZipFile(f,"r") as zip_ref:
            zip_ref.extractall(extract_path)
    

In [25]:
#Folders needed to unzip
sen_zip_folders = create_zip_folders(s_paths,'is')
hr_zip_folders = create_zip_folders(hr_paths,'ih')

In [26]:
#Unzip text folders
#Senate
unzip_text_folder(sen_zip_folders)

#House
unzip_text_folder(hr_zip_folders)

# Parse HTM files into dictionary
***

In [27]:
def bs4_process(s):
    soup = BeautifulSoup(open(s))
    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out
    # get text
    text = soup.get_text()
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = ' '.join(chunk for chunk in chunks if chunk)
    #Remove everything before 'A BILL'
    text = text.split('A BILL')[1]
    
    #Remove <all>
    text = text.split('<all>')[0]

    #Remove characters using regex
    #Remove ` character (does not need to be replaced by a space)
    text1 = re.sub('`', '', text)
    #Remove -- character (need to be replaced by a space)
    text2 = re.sub('-{2}', ' ', text1)
    
    #Define patterns
    #'\([^\d]{1}\)' = single non-number character inside parentheses
    #'\((\d+)\)' = numbers 1 or more inside parentheses
    #'[\(MDCLXVI]+\)' = roman numerals inside parentheses
    text3 = re.sub('\([^\d]{1}\)|\((\d+)\)|[\(MDCLXVI]+\)', '', text2, flags=re.IGNORECASE)
    
    #Lowercase
    text3 = text3.lower()
    return text3

In [28]:
#version codes to exclude
version_codes=['as', 'ash', 'ath', 'ats', 'cdh', 'cds', 'cph', 'cps', 'eah', 'eas', 'eh', 'eph', 'enr',
               'es', 'fah', 'fph', 'fps', 'hdh', 'hds', 'iph', 'ips', 'lth', 'lts','oph', 'ops', 'pav',
               'pch', 'pcs', 'pp', 'pap', 'pwah', 'rah', 'ras', 'rch', 'rcs', 'rdh', 'rds', 'reah', 'res',
               'renr', 'rfh', 'rfs', 'rh', 'rih', 'ris', 'rs', 'rth', 'rts', 'sas', 'sc']

#bill formats to exclude
formats=['pdf','xml']

#final exclude list
exclude_list = version_codes+formats

In [29]:
#Function to traverse directory and load jHTM file

def load_htm(paths):
    text_dict={}
    exclude=set(exclude_list) #make sure all version codes are in here
    if len(paths)
    for p in paths:
        for root, dirs, files in os.walk(p, topdown=True):
            dirs[:] = [d for d in dirs if d not in exclude]
            for f in files:
                if f.endswith('.htm'):
                    try:
                        t = bs4_process(root+'\\'+f)
                        k = re.search('BILLS-(.+?)\\\\html', root).group(1)
                        text_dict.update({k:t})
                    except IndexError:
                        print('Bill text is corrupted or incomplete.')
                        pass
                    

    return text_dict

## Senate 107-114th Congress

In [30]:
senate107_114 = load_htm(s_paths[:-1])

Bill text is corrupted or incomplete.
Bill text is corrupted or incomplete.
Bill text is corrupted or incomplete.
Bill text is corrupted or incomplete.


In [31]:
len(senate107_114)

27198

In [34]:
os.chdir('../../IntroMLProject/Data')

In [37]:
#Save to json file

with open('./107th-112th Congress/Bill text/Sen_text_to_114.json', 'w') as outfile:
    json.dump(senate107_114, outfile)


## House 107-114th Congress

In [38]:
HR107_114 = load_htm(hr_paths[:-1])

Bill text is corrupted or incomplete.
Bill text is corrupted or incomplete.
Bill text is corrupted or incomplete.
Bill text is corrupted or incomplete.
Bill text is corrupted or incomplete.


In [39]:
len(HR107_114)

50367

In [40]:
#Save to json file

with open('./107th-112th Congress/Bill text/HR_text_to_114.json', 'w') as outfile:
    json.dump(HR107_114, outfile)

## Testing data: 115th Congress

In [51]:
senate_115 = load_htm([s_paths[-1]])

In [53]:
HR_115 = load_htm([hr_paths[-1]])

Bill text is corrupted or incomplete.


In [54]:
with open('./Testing data 115th Congress/Bill text/senate_115.json', 'w') as outfile:
    json.dump(senate_115, outfile)

In [55]:
with open('./Testing data 115th Congress/Bill text/HR_115.json', 'w') as outfile:
    json.dump(HR_115, outfile)